# Bibliothèques

In [1]:
import pandas as pd
import requests
import mysql.connector
import pymysql

# Import des fichiers employées

In [2]:
def download_google_sheet(file_id, filename):
    url = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx'
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Téléchargé : {filename}")
    else:
        print(f"Échec du téléchargement : {filename} (Status code: {response.status_code})")

# IDs des fichiers Google Sheets
file_ids = {
    'employees-1': '1jQvzhk6sdmHQEdeGkq6a45PHQC7VwIty',
    'employees-2': '1rDcqvGmRvQD_4tZFlL1X345FiCGhrb_d'
}

# Télécharger tous les fichiers
for filename, file_id in file_ids.items():
    download_google_sheet(file_id, f'{filename}.xlsx')

# Chargement des fichiers dans pandas
df1 = pd.read_excel('employees-1.xlsx')
df2 = pd.read_excel('employees-2.xlsx')

Téléchargé : employees-1.xlsx
Téléchargé : employees-2.xlsx


Une vue des 5 premières lignes de données de 'employees-1.xlsx' et 'employees-2.xlsx'

In [3]:
df1.head()

,Nom,Email,Téléphone,Département,Salaire
0,Gabrielle Vaillant,gabrielle.vaillant@exemple.com,+236 5353930534,Finance,911213
1,Guy Fontaine,guy.fontaine@exemple.com,+229 4491809949,Finance,681965
2,Daniel Sauvage,daniel.sauvage@exemple.com,+229 2680672171,Finance,513363
3,Camille Peron,camille.peron@exemple.com,+228 0527054290,Ressources Humaines,1360315
4,Colette Morin,colette.morin@exemple.com,+237 6356844720,Ventes,975207


In [4]:
df2.head()

,Nom,Email,Phone,Poste,Salaire
0,Denis Antoine,denis.antoine@exemple.com,+225 9188720481,Chargé de communication,1156570
1,Renée Lemaire,renée.lemaire@exemple.com,+229 1027159658,Responsable Marketing,1604314
2,Astrid Raynaud,astrid.raynaud@exemple.com,+237 6190825296,Consultant IT,1224586
3,Virginie Hardy,virginie.hardy@exemple.com,+229 7296271141,Responsable RH,506492
4,Célina Morvan,célina.morvan@exemple.com,+225 2588877046,Designer UI/UX,362123


Correspondance des noms de colonnes entre df2 et df1

In [5]:
df2.rename(columns={'Phone': 'Téléphone'}, inplace=True)

# Nettoyage des données

combien de lignes dans df1 ?

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nom          50 non-null     object
 1   Email        50 non-null     object
 2   Téléphone    50 non-null     object
 3   Département  50 non-null     object
 4   Salaire      50 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


combien de lignes dans df2 ?

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nom        50 non-null     object
 1   Email      50 non-null     object
 2   Téléphone  50 non-null     object
 3   Poste      50 non-null     object
 4   Salaire    50 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


liste de tous les départements

In [8]:
dpt_list = df1['Département'].unique().tolist()
dpt_list.sort()
dpt_list

['Direction',
 'Développement',
 'Finance',
 'Juridique',
 'Logistique',
 'Marketing',
 'Ressources Humaines',
 'Support Client',
 'Ventes']

Nombre de départements

In [9]:
len(dpt_list)

9

liste de tous les postes

In [10]:
pst_list = df2['Poste'].unique().tolist()
pst_list.sort()
pst_list

['Analyste Financier',
 'Chargé de communication',
 'Chef de projet',
 'Consultant IT',
 'Designer UI/UX',
 'Développeur Back-end',
 'Développeur Front-end',
 'Responsable Marketing',
 'Responsable RH',
 'Technicien réseau']

Nombre de postes distincts

In [11]:
len(pst_list)

10

Relation Poste - Département

In [12]:
pst_dpt = {
    'Analyste Financier': 'Finance',
    'Chargé de communication': 'Marketing',
    'Chef de projet': 'Développement',
    'Consultant IT': 'Support Client',
    'Designer UI/UX': 'Marketing',
    'Développeur Back-end': 'Développement',
    'Développeur Front-end': 'Développement',
    'Responsable Marketing': 'Marketing',
    'Responsable RH': 'Ressources Humaines',
    'Technicien réseau': 'Logistique'
}

In [13]:
# Ajouter la colonne 'Poste' à df1 avec des NaN
df1['Poste'] = pd.NA

# Ajouter la colonne 'Département' à df2 avec des NaN
df2['Département'] = pd.NA

# Réalignement des colonnes
df1 = df1[['Nom', 'Email', 'Téléphone', 'Département', 'Poste', 'Salaire']]
df2 = df2[['Nom', 'Email', 'Téléphone', 'Département', 'Poste', 'Salaire']]

# Concaténation des deux DataFrames
df3 = pd.concat([df1, df2], ignore_index=True)

# Tri par la colonne 'Nom'
df3.sort_values(by='Nom', inplace=True)

# Réinitialisation de l'index
df3.reset_index(drop=True, inplace=True)

# Affichage dataframe final
df3

,Nom,Email,Téléphone,Département,Poste,Salaire
0,Adrienne Collet,adrienne.collet@exemple.com,+229 4426194133,Ventes,NaN,1799452
1,Adrienne Pichon,adrienne.pichon@exemple.com,+225 4013697528,NaN,Responsable Marketing,1025841
2,Adrienne Richard,adrienne.richard@exemple.com,+241 4519250045,NaN,Développeur Back-end,309901
3,Adèle Benard,adèle.benard@exemple.com,+225 7506509473,NaN,Chargé de communication,1745781
4,Agathe Gilbert,agathe.gilbert@exemple.com,+228 0189816250,Direction,NaN,747985
...,...,...,...,...,...,...
95,Édith Aubry,édith.aubry@exemple.com,+241 8448235598,Ressources Humaines,NaN,1992428
96,Édouard Julien,édouard.julien@exemple.com,+228 0644006957,Finance,NaN,254005
97,Éléonore Loiseau,éléonore.loiseau@exemple.com,+229 7251772581,Logistique,NaN,1833002
98,Émilie Sauvage,émilie.sauvage@exemple.com,+225 9512510859,Juridique,NaN,933949


Combien de noms distincts ?

In [14]:
len(df3['Nom'].unique().tolist())

100

Tous les postes sont attachés à un département

In [15]:
for i in range(len(df3)):
    if df3['Poste'].at[i] in list(pst_dpt.keys()):
        df3['Département'].at[i] = pst_dpt[df3['Poste'].at[i]]

Un département peut avoir plusieurs postes donc les postes inconnus sont marqués 'Inconnu'

In [16]:
# df3['Poste'] = df3['Poste'].fillna('Non spécifié')

Pour ne pas biaiser l'analyse, vu qu'il y a peu de données je vais attribuer des postes à ceux dont les postes sont non spécifiés, normalement dans une entreprise chaque employé a un poste et est rattaché à un département.

In [17]:
pd.set_option('display.max_rows', None)
df3

,Nom,Email,Téléphone,Département,Poste,Salaire
0,Adrienne Collet,adrienne.collet@exemple.com,+229 4426194133,Ventes,NaN,1799452
1,Adrienne Pichon,adrienne.pichon@exemple.com,+225 4013697528,Marketing,Responsable Marketing,1025841
2,Adrienne Richard,adrienne.richard@exemple.com,+241 4519250045,Développement,Développeur Back-end,309901
3,Adèle Benard,adèle.benard@exemple.com,+225 7506509473,Marketing,Chargé de communication,1745781
4,Agathe Gilbert,agathe.gilbert@exemple.com,+228 0189816250,Direction,NaN,747985
5,Aimé De Sousa,aimé.de.sousa@exemple.com,+225 3980536875,Développement,Développeur Front-end,394895
6,Aimée Laine,aimée.laine@exemple.com,+241 2696185705,Marketing,Responsable Marketing,1752593
7,Alexandrie Ferrand,alexandrie.ferrand@exemple.com,+241 9398049708,Juridique,NaN,743621
8,Alice Bourgeois,alice.bourgeois@exemple.com,+225 5974752860,Logistique,Technicien réseau,373022
9,Anastasie Barthelemy,anastasie.barthelemy@exemple.com,+229 2809517660,Développement,Chef de projet,1183984


In [18]:
df3['Poste'].at[0] = 'Responsable des ventes'
df3['Poste'].at[4] = 'Conseiller du directeur'
df3['Poste'].at[7] = 'juriste'
df3['Poste'].at[10] = 'Développeur Front-end'
df3['Poste'].at[15] = 'Responsable juridique'
df3['Poste'].at[16] = 'Directeur Général adjoint'
df3['Poste'].at[20] = 'Responsable juridique adjoint'
df3['Poste'].at[21] = 'Responsable RH'
df3['Poste'].at[24] = 'Responsable avant-vente'
df3['Poste'].at[25] = 'Gestionnaire relation client'
df3['Poste'].at[28] = 'Agent logistique'
df3['Poste'].at[29] = 'Comptable'
df3['Poste'].at[33] = 'Analyste marketing'
df3['Poste'].at[36] = 'Auditeur interne'
df3['Poste'].at[37] = 'Directeur financier'
df3['Poste'].at[38] = 'Consultant RH'
df3['Poste'].at[39] = 'Gestionnaire de paie'
df3['Poste'].at[40] = 'Designer UI/UX'
df3['Poste'].at[42] = 'Juriste'
df3['Poste'].at[43] = 'Directeur juridique'
df3['Poste'].at[44] = 'Assistante juridique'
df3['Poste'].at[45] = 'Gestionnaire relation client'
df3['Poste'].at[46] = 'Directeur RH'
df3['Poste'].at[47] = 'Responsable logistique'
df3['Poste'].at[48] = 'Commerciale'
df3['Poste'].at[49] = 'Directeur Logistique'
df3['Poste'].at[51] = 'Chef de secteur'
df3['Poste'].at[54] = 'Assistante commerciale'
df3['Poste'].at[57] = 'Consultant RH'
df3['Poste'].at[60] = 'Directeur Technique adjoint'
df3['Poste'].at[61] = 'Directeur Logistique'
df3['Poste'].at[63] = 'Gestionnaire relation client'
df3['Poste'].at[66] = 'Responsable de la satisfaction client'
df3['Poste'].at[67] = 'Manager du support client'
df3['Poste'].at[68] = 'Assistante'
df3['Poste'].at[69] = 'Juriste'
df3['Poste'].at[72] = 'Manager adjoint du support client'
df3['Poste'].at[73] = 'Cheffe de projet'
df3['Poste'].at[75] = 'Juriste'
df3['Poste'].at[79] = 'Gestionnaire de la chaîne d\'approvisionnement'
df3['Poste'].at[81] = 'Juriste'
df3['Poste'].at[85] = 'Consultant RH'
df3['Poste'].at[88] = 'Consultant RH'
df3['Poste'].at[90] = 'Designer UI/UX'
df3['Poste'].at[92] = 'Responsable logistique adjoint'
df3['Poste'].at[95] = 'Consultant Sénior RH'
df3['Poste'].at[96] = 'Assistant'
df3['Poste'].at[97] = 'Responsable logistique nuit'
df3['Poste'].at[98] = 'Juriste'
df3['Poste'].at[99] = 'Développeur Front-end'

Suppression de tout espace superflu situé avant ou après les chaines de caractères

In [19]:
df3['Nom'] = df3['Nom'].str.strip()
df3['Email'] = df3['Email'].str.strip()
df3['Téléphone'] = df3['Téléphone'].str.strip()
df3['Département'] = df3['Département'].str.strip()
df3['Poste'] = df3['Poste'].str.strip()

In [20]:
df3

,Nom,Email,Téléphone,Département,Poste,Salaire
0,Adrienne Collet,adrienne.collet@exemple.com,+229 4426194133,Ventes,Responsable des ventes,1799452
1,Adrienne Pichon,adrienne.pichon@exemple.com,+225 4013697528,Marketing,Responsable Marketing,1025841
2,Adrienne Richard,adrienne.richard@exemple.com,+241 4519250045,Développement,Développeur Back-end,309901
3,Adèle Benard,adèle.benard@exemple.com,+225 7506509473,Marketing,Chargé de communication,1745781
4,Agathe Gilbert,agathe.gilbert@exemple.com,+228 0189816250,Direction,Conseiller du directeur,747985
5,Aimé De Sousa,aimé.de.sousa@exemple.com,+225 3980536875,Développement,Développeur Front-end,394895
6,Aimée Laine,aimée.laine@exemple.com,+241 2696185705,Marketing,Responsable Marketing,1752593
7,Alexandrie Ferrand,alexandrie.ferrand@exemple.com,+241 9398049708,Juridique,juriste,743621
8,Alice Bourgeois,alice.bourgeois@exemple.com,+225 5974752860,Logistique,Technicien réseau,373022
9,Anastasie Barthelemy,anastasie.barthelemy@exemple.com,+229 2809517660,Développement,Chef de projet,1183984


Création d'une colonne pays basé sur l'indicatif

Regex pour extraire l'indicatif

In [21]:
df3['Indicatif'] = df3['Téléphone'].str.extract(r'(\+\d{1,3})')[0]

Dictionnaire pour association indicatif - pays

In [22]:
indicatif = {
    '+229': 'Bénin',
    '+225': 'Côte d\'Ivoire',
    '+241': 'Gabon',
    '+228': 'Togo',
    '+237': 'Cameroun',
    '+236': 'Centrafique',
}

In [23]:
indic_list = list(df3['Indicatif'].unique())

Création d'une colonne vide nommée 'Pays'

In [24]:
# Ajouter la colonne 'Pays' à df3 avec des NaN
df3['Pays'] = pd.NA

In [25]:
for i in range(len(df3)):
    if df3['Indicatif'].at[i] in indic_list:
        df3['Pays'].at[i] = indicatif[df3['Indicatif'].at[i]]

Suppression colonne 'Indicatif'

In [26]:
df3.drop('Indicatif', axis=1, inplace=True)

Export clean en parquet et csv

In [27]:
df3.to_parquet('cleaned_employees.parquet', index=False)
df3.to_csv('cleaned_employees.csv', index=False)
print("Fichier nettoyé et sauvegardé en CSV et Parquet.")

Fichier nettoyé et sauvegardé en CSV et Parquet.


# Amazon RDS

In [ ]:
# Connexion à votre base RDS
db_host = ''
db_name = ''
db_user = ''
db_password = ''

connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             database=db_name)

print("Connected to the database successfully.")

cursor = connection.cursor()

# Exécuter la commande pour lister toutes les bases
cursor.execute("SHOW DATABASES;")

# Récupérer toutes les bases
databases = cursor.fetchall()
print("Databases:", databases)

# Si vous souhaitez connaître la version du serveur MySQL
cursor.execute("SELECT VERSION();")
db_version = cursor.fetchone()
print("Database version:", db_version[0])

# Fermer le curseur et la connexion
cursor.close()
connection.close()
print("Connection closed.")

Connected to the database successfully.
Databases: (('employees_cleaned',), ('information_schema',), ('mysql',), ('performance_schema',), ('sys',))
Database version: 8.0.41
Connection closed.


In [ ]:
# Connexion à la base
conn = mysql.connector.connect(
    host= '',
    user='',
    password='',
    database=''
)

cursor = conn.cursor()

# Requête SQL pour créer la table
create_table_query = """
CREATE TABLE IF NOT EXISTS employees_codon (
  id INT AUTO_INCREMENT PRIMARY KEY,
  Nom VARCHAR(255),
  Email VARCHAR(255) UNIQUE,
  Téléphone VARCHAR(50),
  Département VARCHAR(100),
  Poste VARCHAR(100),
  Salaire INT,
  Pays VARCHAR(100)
);
"""

cursor.execute(create_table_query)

# Valider
conn.commit()

cursor.close()
conn.close()

print("Table 'employees_codon' créée avec succès.")

Table 'employees_codon' créée avec succès.


In [ ]:
# Connexion à la base
conn = mysql.connector.connect(
    host= '',
    user='',
    password='',
    database=''
)

cursor = conn.cursor()

# Préparer la liste des tuples, chaque tuple correspondant à une ligne
data_tuples = [
    (
        row['Nom'],
        row['Email'],
        row['Téléphone'],
        row['Département'],
        row['Poste'],
        row['Salaire'],
        row['Pays']
    )
    for _, row in df3.iterrows()
]

# Requête d'insertion
insert_query = """
    INSERT INTO employees_codon (Nom, Email, Téléphone, Département, Poste, Salaire, Pays)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        Nom=VALUES(Nom),
        Téléphone=VALUES(Téléphone),
        Département=VALUES(Département),
        Poste=VALUES(Poste),
        Salaire=VALUES(Salaire),
        Pays=VALUES(Pays)
"""

# Supprimer toutes les données
cursor.execute("DELETE FROM employees_codon;")

# Réinitialiser l'auto-incrémentation
cursor.execute("ALTER TABLE employees_codon AUTO_INCREMENT = 1;")

# Insérer toutes les lignes en une seule opération
cursor.executemany(insert_query, data_tuples)

# Valider la transaction
conn.commit()

# Fermer la connexion
cursor.close()
conn.close()

print("Toutes les données ont été insérées dans la table 'employees_codon'.")

Toutes les données ont été insérées dans la table 'employees_codon'.


In [ ]:
# Connexion à la base
conn = mysql.connector.connect(
    host= '',
    user='',
    password='',
    database=''
)

cursor = conn.cursor()

# Récupérer les 10 premières lignes
cursor.execute("SELECT * FROM employees_codon LIMIT 10;")
rows = cursor.fetchall()

# Afficher
for row in rows:
    print(row)

cursor.close()
conn.close()

(1, 'Adrienne Collet', 'adrienne.collet@exemple.com', '+229 4426194133', 'Ventes', 'Responsable des ventes', 1799452, 'Bénin')
(2, 'Adrienne Pichon', 'adrienne.pichon@exemple.com', '+225 4013697528', 'Marketing', 'Responsable Marketing', 1025841, "Côte d'Ivoire")
(3, 'Adrienne Richard', 'adrienne.richard@exemple.com', '+241 4519250045', 'Développement', 'Développeur Back-end', 309901, 'Gabon')
(4, 'Adèle Benard', 'adèle.benard@exemple.com', '+225 7506509473', 'Marketing', 'Chargé de communication', 1745781, "Côte d'Ivoire")
(5, 'Agathe Gilbert', 'agathe.gilbert@exemple.com', '+228 0189816250', 'Direction', 'Conseiller du directeur', 747985, 'Togo')
(6, 'Aimé De Sousa', 'aimé.de.sousa@exemple.com', '+225 3980536875', 'Développement', 'Développeur Front-end', 394895, "Côte d'Ivoire")
(7, 'Aimée Laine', 'aimée.laine@exemple.com', '+241 2696185705', 'Marketing', 'Responsable Marketing', 1752593, 'Gabon')
(8, 'Alexandrie Ferrand', 'alexandrie.ferrand@exemple.com', '+241 9398049708', 'Juridi

In [ ]:
# Connexion à la base
conn = mysql.connector.connect(
    host= '',
    user='',
    password='',
    database=''
)

cursor = conn.cursor()

# nombre de lignes dans la table
cursor.execute("SELECT COUNT(*) FROM employees_codon;")
count = cursor.fetchone()
print(f"Nombre de lignes dans la table : {count[0]}")

cursor.close()
conn.close()

Nombre de lignes dans la table : 100
